In [1]:
import cv2
import numpy as np
from keras.models import load_model

# Load models
age_model = load_model('age_model.h5', compile=False)
emotion_model = load_model('emotion_model.h5', compile=False)

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Start video capture
cap = cv2.VideoCapture(0)

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]

        # -------- Age Prediction --------
        face_img_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        face_img_resized_age = cv2.resize(face_img_rgb, (100, 100)) / 255.0
        face_img_resized_age = np.expand_dims(face_img_resized_age, axis=0)
        predicted_age = age_model.predict(face_img_resized_age)
        predicted_age = int(predicted_age[0][0])

        # -------- Emotion Prediction --------
        face_img_resized_emotion = cv2.resize(face_img, (100, 100)) / 255.0  # Resize to 100x100
        face_img_resized_emotion = np.expand_dims(face_img_resized_emotion, axis=0)  # Add batch dimension
        emotion_prediction = emotion_model.predict(face_img_resized_emotion)
        emotion_label = emotion_labels[np.argmax(emotion_prediction)]

        # -------- Drawing Box and Info --------
        if predicted_age < 13 or predicted_age > 60:
            box_color = (0, 0, 255)  # Red
            text = f"Age: {predicted_age} (Not Allowed 🚫)"
        else:
            box_color = (0, 255, 0)  # Green
            text = f"Age: {predicted_age} (Allowed ✅)"

        cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
        cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)
        cv2.putText(frame, f"Emotion: {emotion_label}", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow('Age and Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━

In [2]:
# Import necessary libraries
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming your test dataset is X_test and y_test
# And your model is called 'model'

# 1. Get model predictions
y_pred_probs = model.predict(X_test)  # Predict probabilities

# If you have multi-class classification
y_pred_classes = np.argmax(y_pred_probs, axis=1)  
y_true_classes = np.argmax(y_test, axis=1)

# 2. Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plotting the confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Precision and Recall
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Precision Score: {precision:.4f}")
print(f"Recall Score: {recall:.4f}")

# 4. Full Classification Report (optional but very good)
print("\nClassification Report:\n")
print(classification_report(y_true_classes, y_pred_classes))


NameError: name 'model' is not defined

In [1]:
# Import necessary libraries
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 1. Get model predictions
y_pred_probs = model.predict(X_test)  # Predict probabilities

# If you have multi-class classification
y_pred_classes = np.argmax(y_pred_probs, axis=1)  
y_true_classes = np.argmax(y_test, axis=1)

# 2. Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plotting the confusion matrix
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Precision and Recall
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Precision Score: {precision:.4f}")
print(f"Recall Score: {recall:.4f}")

# 4. Full Classification Report (optional but very good)
print("\nClassification Report:\n")
print(classification_report(y_true_classes, y_pred_classes))


NameError: name 'model' is not defined

In [2]:
# 📦 Import necessary libraries
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 📈 1. Predict probabilities on test set
y_pred_probs = model.predict(X_test)

# 🏷️ 2. Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # If one-hot encoded
y_true_classes = np.argmax(y_test, axis=1)

# 🧩 3. Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 🎯 4. Precision and Recall
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Precision Score: {precision:.4f}")
print(f"Recall Score: {recall:.4f}")

# 🏆 5. Full Classification Report
print("\nClassification Report:\n")
print(classification_report(y_true_classes, y_pred_classes))


NameError: name 'model' is not defined